In [1]:
import numpy as np
import pandas as pd
from natsort import natsorted
from glob import glob
import nltk
import os

In [ ]:
n_w = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10','11', '12', '13','14', '15' ,'17', '18',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','K', 'L', 'M', 'N']
base_path =   os.path.expanduser('~/Box/COVID-19 Adolphs Lab/')
data_dir = base_path+ '/PreProcessed_Data/'

In [ ]:
data = pd.read_csv(data_dir + 'Wave1-17_A-N_release.csv',encoding = 'ISO-8859-1',dtype=str, keep_default_na=False, na_values=['','NA'], low_memory=False)
#data.PROLIFIC_PID = data.PROLIFIC_PID.str.strip('- ')
print(sum(data.PROLIFIC_PID.str.endswith('-')))
data = data.reindex(natsorted(data.columns), axis=1)
data = data.loc[~data['PROLIFIC_PID'].isna()]

attent_vars = ['NIHE1_9', 'DemC25', 'DISG1.2_23', 'DISG1.1_23', 'DISG2.2_23', 'ReSe1_23', 'Fed_13', 'RW23', 'GFPS2_11', 'EES1_32']


# Pipeline to create the validation file

### 1. Time

In [ ]:
validation = data.loc[:,['PROLIFIC_PID', 'wave']]

In [ ]:
s = data.loc[:,'V4'].apply(pd.Timestamp)-data.loc[:,'V3'].apply(pd.Timestamp)

In [ ]:
validation.loc[:,'total_time'] = s.apply(lambda x: x.seconds/60)

## 2. Attention check questions

In [ ]:
# Task
validation['task_completed'] = data['Tsk4']

## 4. response string length

In [ ]:

def long_string(row):
    # Return a list of response string length
    # The calculation of MaxLongString, meanLongString and countLongString can be calculated from this list
    count_ls = []
    string_len = 1
    for i in range(1, len(row)):
        #print(i)
        if np.isnan(float(row[i])):
            # If the row contains all nas, then will append an empty list
#             if string_len > 1:
#                 count_ls.append(string_len)
#                 string_len = 1
            continue
        elif row[i] == row[i-1]:
            string_len += 1
        elif row[i-1] is not np.nan:
            count_ls.append(string_len)
            string_len = 1
    
    if ~np.all(np.isnan([float(x) for x in row])):
        count_ls.append(string_len)
    return np.array(count_ls)


def get_max(ls):
    if ls.size == 0:
        return np.nan
    else:
        return ls[np.argmax(ls)]
    
    
def get_mean(ls):
    if ls.size == 0:
        return np.nan
    else:
        return np.mean(ls)
    
def count_greater_than_5(ls):
    if ls.size == 0:
        return np.nan
    else:
        return np.sum(ls >= 5)
    
def response_string(dat, name, validation):
    s_long_string = dat.apply(long_string, axis=1)
    validation['maxLongString_'+name] = s_long_string.apply(get_max)
    validation['meanLongString_'+name] = s_long_string.apply(get_mean)
    validation['countLongString_'+name] = s_long_string.apply(count_greater_than_5)
    
    return validation, s_long_string, ['maxLongString_'+name, 'meanLongString_'+name, 'countLongString_'+name]

def inspect(dat, validation, s_long_string, input_vars, output_vars, inspect_w='1'):
    print(dat.loc[dat['wave']==inspect_w, input_vars].head())
    print(s_long_string[validation['wave']==inspect_w].head())
    print(validation.loc[validation['wave']==inspect_w, output_vars].head())


def process_na(dat, var_names):
    for w in np.unique(dat['wave']):
        for var in var_names:
            if np.sum(~dat.loc[dat['wave']==w, var].isna())>0:
                dat.loc[dat['wave']==w,var] = dat.loc[dat['wave']==w, var].fillna(0)
        
#         if((np.sum(np.sum(~dat.loc[dat['wave']==w, var_names].isna()))) > 0) and flag:
#             dat.loc[dat['wave']==w, var_names] = dat.loc[dat['wave']==w, var_names].fillna(0)
        
    return dat

def run_analysis(data, validation, name, input_vars, inspect_w=''):
    data = process_na(data, input_vars)
    input_dat = data.loc[:, input_vars]
    validation, s, output_vars = response_string(input_dat, name, validation)
    
    if inspect_w:
        inspect(data, validation, s, input_vars, output_vars, inspect_w=inspect_w)
    
    return validation, data


def construct_vars(prefix, n, postfix=None, skip=[]):
    ls = []
    for i in range(n):
        if (i+1) not in skip:
            out = prefix + str(i+1)
            if postfix:
                out = out + postfix
            ls.append(out)
    return ls
    

In [ ]:
name = 'EPII_infectHist_no'
input_vars = 
validation, data = run_analysis(data, validation, name, input_vars)

In [ ]:
name = 'EPII_infectHist_NA'
input_vars = 
validation, data = run_analysis(data, validation, name, input_vars)

In [1]:
name = 'EPII_posChange_you'
input_vars = 
print(input_vars)
validation, data = run_analysis(data, validation, name, input_vars)

NameError: name 'construct_vars' is not defined

## 6. Calculate interquartile range


In [ ]:
def interq_range(s, n):
    # calculate the intequartile range given a pandas series - a list of responses
    s_cleaned = s[~s.isna()]
    if len(s_cleaned) == 0:
        return [np.nan, np.nan]
    else:
        q1 = np.quantile(s_cleaned.astype(float), 0.25)
        q3 = np.quantile(s_cleaned.astype(float), 0.75)
        return [q1-n*(q3-q1), q3+n*(q3-q1)]

def interq_analysis_old(dat, n_w, n_q, validation):
    dat_tmp = dat.copy()
    for w in n_w:
        interq_r = dat.loc[validation['wave'] == w].apply(lambda x: interq_range(x, 1), axis=0)
        print(interq_r)
        for col in list(interq_r.index):
            dat_no_na = dat.loc[(validation['wave'] == w) & (~dat[col].isna())]
            if ~np.any(np.isnan(interq_r[col])):
                idx = (validation['wave'] == w) & (dat_no_na[col]<=interq_r[col][1]) & (dat_no_na[col]>=interq_r[col][0])
                dat_tmp.loc[idx, col] = 0

    for q in range(n_q):
        for w in n_w:
            interq_r = dat.loc[validation['wave'] == w].apply(lambda x: interq_range(x, q+1), axis=0)
            for col in list(interq_r.index):
                dat_no_na = dat.loc[(validation['wave'] == w) & (~dat[col].isna())]
                if ~np.any(np.isnan(interq_r[col])):
                    idx = (validation['wave'] == w) & ((dat_no_na[col]<interq_r[col][0]) | (dat_no_na[col]>interq_r[col][1]))
                    dat_tmp.loc[idx, col] = q+1
    return dat_tmp


def interq_analysis(dat, n_w, n_q, validation):
    dat_tmp = dat.copy()
    for w in n_w:
        interq_r = dat.loc[validation['wave'] == w].apply(lambda x: interq_range(x, 1), axis=0)
        for col in list(interq_r.columns):
            dat_no_na = dat.loc[(validation['wave'] == w) & (~dat[col].isna())]
            if ~np.any(np.isnan(interq_r[col])):
                dat_no_na.head
                idx = (validation['wave'] == w) & (dat_no_na[col]<=interq_r[col][1]) & (dat_no_na[col]>=interq_r[col][0])
                dat_tmp.loc[idx, col] = 0

    for q in range(n_q):
        for w in n_w:
            interq_r = dat.loc[validation['wave'] == w].apply(lambda x: interq_range(x, q+1), axis=0)
            for col in list(interq_r.columns):
                dat_no_na = dat.loc[(validation['wave'] == w) & (~dat[col].isna())]
                if ~np.any(np.isnan(interq_r[col])):
                    idx = (validation['wave'] == w) & ((dat_no_na[col]<interq_r[col][0]) | (dat_no_na[col]>interq_r[col][1]))
                    dat_tmp.loc[idx, col] = q+1
    return dat_tmp

In [ ]:
rs_mean = validation.filter(regex='meanLongString', axis=1)
interq_rs_mean = interq_analysis(rs_mean, n_w, 3, validation)

rs_max = validation.filter(regex='maxLongString', axis=1)
interq_rs_max = interq_analysis(rs_max, n_w, 3, validation)

interq_rs = pd.concat([interq_rs_mean, interq_rs_max], axis=1)
interq_rs['wave'] = data['wave']
interq_rs['PROLIFIC_PID'] = data['PROLIFIC_PID']

# 7. Additional IQR variables

In [ ]:
add_data = pd.read_csv(base_path+'PreProcessed_Data/validation_files/additional_IQRmeasures_w1-17_wA-N.csv', index_col=0)
p_dat = add_data.drop(['PROLIFIC_PID', 'wave'], axis=1)
interq_add = interq_analysis(p_dat, n_w, 3, add_data)
interq_add['wave'] = add_data['wave']
interq_add['PROLIFIC_PID'] = add_data['PROLIFIC_PID']
all_interq = interq_rs.merge(interq_add, on=['PROLIFIC_PID', 'wave'], how='left')
interq_add


In [ ]:
task_dat = pd.read_csv(base_path+'/PreProcessed_Data/validation_files/all_tasks_w1-17_A-N.csv',dtype=str, keep_default_na=False, na_values=['','nan'])
miss_cols = list(task_dat.columns[task_dat.columns.str.endswith('_missing')])
admin_cols = list(task_dat.columns[task_dat.columns.str.endswith( '_administered')])

p_task_dat = task_dat[['tr_1s_rt_pctlt_300','tr_1s_medianRT','tr_1s_meanLongString','amp_pct_bad_rts', 'amp_medianRT','amp_meanLongString', 
        'altt_meanLongString', 'altt_medianRT', 'cvd_consp_meanLongString']]
p_task_dat = p_task_dat.astype(float)
interq_task = interq_analysis(p_task_dat, n_w, 3, task_dat)
interq_task['wave'] = task_dat['wave']
interq_task['PROLIFIC_PID'] = task_dat['PROLIFIC_PID']
all_interq = all_interq.merge(interq_task, on=['PROLIFIC_PID', 'wave'], how='left')
all_interq



task_dat

In [ ]:
#all_interq = interq_rs.merge(interq_add, on=['PROLIFIC_PID', 'wave'], how='left')
all_interq.to_csv(base_path+'/PreProcessed_Data/validation_files/all_IQRmeasures_output_w1-17_A-N.csv', index=None)
